In [5]:
import psycopg2

In [7]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=postgres password=student")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print(e)

conn.set_session(autocommit=True)

In [28]:
cur.execute("DROP TABLE IF EXISTS fact_customer_transactions")
cur.execute("DROP TABLE IF EXISTS dimension_customer")
cur.execute("DROP TABLE IF EXISTS dimension_store")
cur.execute("DROP TABLE IF EXISTS dimension_items_purchased")

In [30]:
# Below DB is in the STAR SCHEMA, if you don't know it, Google for.
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS fact_customer_transactions (customer_id int, store_id int, spent float)")
    cur.execute("CREATE TABLE IF NOT EXISTS dimension_customer (customer_id int, name text, rewards boolean)")
    cur.execute("CREATE TABLE IF NOT EXISTS dimension_store (store_id int, state text)")
    cur.execute("CREATE TABLE IF NOT EXISTS dimension_items_purchased (customer_id int, item_number int, item_name text)")
except psycopg2.Error as e: 
    print (e)
    
#Insert into all tables 
try: 
    cur.execute("INSERT INTO fact_customer_transactions (customer_id, store_id, spent) VALUES \
                  (%s, %s, %s);",(1, 1, 20.50))
    cur.execute("INSERT INTO fact_customer_transactions (customer_id, store_id, spent) VALUES \
                  (%s, %s, %s);",(2, 1, 35.21))
except psycopg2.Error as e: 
    print (e)

try: 
    cur.execute("INSERT INTO dimension_customer (customer_id, name, rewards) VALUES \
                  (%s, %s, %s);",(1, "Amanda", True))
    cur.execute("INSERT INTO dimension_customer (customer_id, name, rewards) VALUES \
                  (%s, %s, %s);",(2, "Toby", False))
except psycopg2.Error as e: 
    print (e)
    
try: 
    cur.execute("INSERT INTO dimension_store (store_id, state) VALUES \
                  (%s, %s);",(1, "CA"))
    cur.execute("INSERT INTO dimension_store (store_id, state) VALUES \
                  (%s, %s);",(2, "WA"))
except psycopg2.Error as e: 
    print (e)

try: 
    cur.execute("INSERT INTO dimension_items_purchased (customer_id, item_number, item_name) VALUES \
                  (%s, %s, %s);",(1, 1, "Rubber Soul"))
    cur.execute("INSERT INTO dimension_items_purchased (customer_id, item_number, item_name) VALUES \
                  (%s, %s, %s);",(2, 3, "Let it Be"))
except psycopg2.Error as e: 
    print (e)


print("Table: fact_customer_transactions")
try: 
    cur.execute("SELECT * FROM fact_customer_transactions")
except psycopg2.Error as e: 
    print (e)

row = cur.fetchone()
while row:
  print(row)
  row = cur.fetchone()

print("Table: dimension_customer")
try: 
    cur.execute("SELECT * FROM dimension_customer")
except psycopg2.Error as e: 
    print (e)

row = cur.fetchone()
while row:
  print(row)
  row = cur.fetchone()

print("Table: dimension_store")
try: 
    cur.execute("SELECT * FROM dimension_store")
except psycopg2.Error as e: 
    print (e)

row = cur.fetchone()
while row:
  print(row)
  row = cur.fetchone()

print("Table: dimension_items_purchased")
try: 
    cur.execute("SELECT * FROM dimension_items_purchased")
except psycopg2.Error as e: 
    print (e)

row = cur.fetchone()
while row:
  print(row)
  row = cur.fetchone()


Table: fact_customer_transactions
(1, 1, 20.5)
(2, 1, 35.21)
Table: dimension_customer
(1, 'Amanda', True)
(2, 'Toby', False)
Table: dimension_store
(1, 'CA')
(2, 'WA')
Table: dimension_items_purchased
(1, 1, 'Rubber Soul')
(2, 3, 'Let it Be')


In [ ]:
# TABLES FORMAT
#    cur.execute("CREATE TABLE IF NOT EXISTS fact_customer_transactions (customer_id int, store_id int, spent float)")
#    cur.execute("CREATE TABLE IF NOT EXISTS dimension_customer (customer_id int, name text, rewards boolean)")
#    cur.execute("CREATE TABLE IF NOT EXISTS dimension_store (store_id int, state text)")
#    cur.execute("CREATE TABLE IF NOT EXISTS dimension_items_purchased (customer_id int, item_number int, item_name text)")


#Now run the following queries on this data easily because of utilizing the Fact/ Dimension and Star Schema
#Query 1: Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.
# select name, store_id, state, item_name, rewards \
#    FROM (fact_customer_transactions 
#       JOIN dimension_customer        ON fact_customer_transactions.customer_id = dimension_customer.customer_id \
#       JOIN dimension_store           ON fact_customer_transactions.store_id = dimension_store.store_id \
#       JOIN dimension_items_purchased ON fact_customer_transactions.customer_id = dimension_items_purchased.customer_id
#    ) WHERE spent > 30.0

#Query 2: How much did Customer 2 spend?
# select SUM(spent) from fact_customer_transactions where fact_customer_transactions.customer_id = 2

In [35]:
print("Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.\n")
try: 
    cur.execute("\
        select name, fact_customer_transactions.store_id, state, item_name, rewards \
          FROM (fact_customer_transactions \
            JOIN dimension_customer        ON fact_customer_transactions.customer_id = dimension_customer.customer_id \
            JOIN dimension_store           ON fact_customer_transactions.store_id = dimension_store.store_id \
            JOIN dimension_items_purchased ON fact_customer_transactions.customer_id = dimension_items_purchased.customer_id \
          ) WHERE spent > 30.0\
    ")
except psycopg2.Error as e: 
    print (e)

row = cur.fetchone()
while row:
  print(row)
  row = cur.fetchone()


Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.

('Toby', 1, 'CA', 'Let it Be', False)


In [39]:
print("How much did Customer 2 spend\n")
try: 
    cur.execute("select customer_id, SUM(spent) from fact_customer_transactions where fact_customer_transactions.customer_id = 2 group by customer_id")
except psycopg2.Error as e: 
    print (e)

row = cur.fetchone()
while row:
  print(row)
  row = cur.fetchone()

How much did Customer 2 spend

(2, 35.21)


In [40]:
cur.execute("DROP TABLE IF EXISTS fact_customer_transactions")
cur.execute("DROP TABLE IF EXISTS dimension_customer")
cur.execute("DROP TABLE IF EXISTS dimension_store")
cur.execute("DROP TABLE IF EXISTS dimension_items_purchased")
cur.close()
conn.close()